In [133]:
import requests
import json
import pandas as pd
import datetime

#extract dividend data
response = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=aapl&outputsize=full&apikey=GZZHIBO1SGTLPGB1')
data = response.text
parsed = json.loads(data)
tsdata = parsed['Time Series (Daily)']
tsda = pd.DataFrame.from_dict(tsdata, orient='index')
tsda_new=tsda.loc[:'2009-01-02',]

In [134]:
#extracting dividends
dividend=tsda_new.drop(['1. open','2. high','3. low','5. adjusted close','6. volume','8. split coefficient'],1)
dividend.index = pd.to_datetime(dividend.index)
dividend.rename(columns={"4. close":"price","7. dividend amount":"adjusted_dividend"},inplace=True)
#cleaning of dividend data with stock split data
dividend.adjusted_dividend = pd.to_numeric(dividend.adjusted_dividend)
dividend.price = pd.to_numeric(dividend.price)
#remove zeroes
indexNames = dividend[ dividend['adjusted_dividend'] == 0 ].index
dividend.drop(indexNames , inplace=True)
#account for stock split
dividend_before_2020=dividend.loc['2020-08-07':,]
dividend_before_2020=(dividend_before_2020)/4
dividend_before_2014=dividend_before_2020.loc['2014-05-08':,]
dividend_before_2014=dividend_before_2014/7
dividend.update(dividend_before_2020)
dividend.update(dividend_before_2014)
dividend['dividend yield'] = dividend['adjusted_dividend']/dividend['price']
print(dividend)

                 price  adjusted_dividend  dividend yield
2021-02-05  136.760000           0.205000        0.001499
2020-11-06  118.690000           0.205000        0.001727
2020-08-07  111.112500           0.205000        0.001845
2020-05-08   77.532500           0.205000        0.002644
2020-02-07   80.007500           0.192500        0.002406
2019-11-07   64.857500           0.192500        0.002968
2019-08-09   50.247500           0.192500        0.003831
2019-05-10   49.295000           0.192500        0.003905
2019-02-08   42.602500           0.182500        0.004284
2018-11-08   52.122500           0.182500        0.003501
2018-08-10   51.882500           0.182500        0.003518
2018-05-11   47.147500           0.182500        0.003871
2018-02-09   39.102500           0.157500        0.004028
2017-11-10   43.667500           0.157500        0.003607
2017-08-10   38.830000           0.157500        0.004056
2017-05-11   38.487500           0.157500        0.004092
2017-02-09   3

In [135]:
#extract interest rates
interest=pd.read_csv('FEDFUNDS.csv',index_col=0)
interest.index = pd.to_datetime(interest.index)

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

key='GZZHIBO1SGTLPGB1'
ts = TimeSeries(key, output_format='pandas')
ti = TechIndicators(key)

aapl_data, aapl_meta_data = ts.get_daily_adjusted(symbol='AAPL',outputsize='full')
aapl_sma,aapl_metadata_sma = ti.get_sma(symbol='AAPL')

figure (num=None, figsize=(15,6), dpi=80)
aapl_data['5. adjusted close'].plot()
plt.tight_layout()
plt.grid
plt.show()
print(aapl_data)

In [ ]:
aapl_time=aapl_data.loc[:'2009-01-02',];
aapl_dividend=aapl_time.drop(['1. open','2. high','3. low','4. close','5. adjusted close','6. volume'],1)
aapl_dividend.rename(columns={"7. dividend amount":"dividend","8. split coefficient":"split"},inplace=True)
indexNames = aapl_dividend[ (aapl_dividend['dividend'] == 0) & (aapl_dividend['split'] == 1.0) ].index
aapl_dividend.drop(indexNames , inplace=True)
aapl_dividend


In [ ]:
import requests
import pandas as pd

# get data from api
base_url = 'https://www.alphavantage.co/query?'
params = {'function': 'TIME_SERIES_DAILY','symbol': 'AAPL', 'apikey': 'GZZHIBO1SGTLPGB1'}
response = requests.get(base_url, params=params)

# convert text data in to a list of of list
data = [row.strip().split(',') for row in response.text.split('\n')]

# load data into a dataframe
#df = pd.DataFrame(data[1:-1][1:-1], columns=data[0])

# display(df)
data

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(interest.index, interest['FEDFUNDS'])
plt.xlabel("Time")
plt.ylabel("Fed Interest")
plt.title("Fed Interest Rates from 2009-2021")

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(dividend.index, dividend['adjusted_dividend'])

plt.xlabel("Time")
plt.ylabel("Dividend")
plt.title("Adjusted Dividend")